In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import time

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
# Increased num_workers for potentially faster data loading if CPU is powerful
# and not bottle-necked by GPU. Adjust based on your system.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4, pin_memory=True if device.type == 'cuda' else False)

100%|██████████| 170M/170M [00:03<00:00, 42.9MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4, pin_memory=True if device.type == 'cuda' else False)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img.cpu() # Move tensor to CPU for numpy conversion
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter) # This will be on CPU by default from DataLoader

# show images
# To show images, they'd need to be moved to CPU if they were on GPU
# For example: imshow(torchvision.utils.make_grid(images.cpu()))
print (labels)
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


tensor([4, 7, 1, 1])
deer  horse car   car  


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Record start time
start_time = time.time()

print("Starting Training...")
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # ---- Move inputs and labels to the chosen device ----
        inputs, labels = data[0].to(device), data[1].to(device)
        # ----------------------------------------------------

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# Record end time
end_time = time.time()

# Calculate and print training time
training_time = end_time - start_time
print(f'Training took: {training_time:.2f} seconds')

Starting Training...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[1,  2000] loss: 2.223
[1,  4000] loss: 1.873
[1,  6000] loss: 1.688
[1,  8000] loss: 1.575
[1, 10000] loss: 1.534
[1, 12000] loss: 1.476


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[2,  2000] loss: 1.434
[2,  4000] loss: 1.379
[2,  6000] loss: 1.349
[2,  8000] loss: 1.329
[2, 10000] loss: 1.292
[2, 12000] loss: 1.316
Finished Training
Training took: 111.64 seconds


In [ ]:

PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
# --- Testing Part ---
# We need a new instance of the network for loading the state dict,
# and it also needs to be on the correct device.
net_loaded = Net()
net_loaded.to(device) # ---- Move the loaded model to the chosen device ----
net_loaded.load_state_dict(torch.load(PATH)) # Load the state dictionary

# Get some test images
dataiter = iter(testloader)
images_test, labels_test = next(dataiter)
images_test, labels_test = images_test.to(device), labels_test.to(device) # Move test data to device

# print images (optional, ensure imshow handles tensors on device or move to cpu)
# imshow(torchvision.utils.make_grid(images_test.cpu()))
# print('GroundTruth: ', ' '.join(f'{classes[labels_test[j]]:5s}' for j in range(batch_size)))


outputs_test = net_loaded(images_test) # Use the loaded and device-mapped network

_, predicted_test = torch.max(outputs_test, 1)

# print('Predicted: ', ' '.join(f'{classes[predicted_test[j]]:5s}'
#                               for j in range(batch_size)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad(): # Disables gradient calculation
    net_loaded.eval() # Set the model to evaluation mode
    for data in testloader:
        # ---- Move inputs and labels to the chosen device ----
        images, labels = data[0].to(device), data[1].to(device)
        # ----------------------------------------------------
        # calculate outputs by running images through the network
        outputs = net_loaded(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy of the network on the 10000 test images: 53 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad(): # Disables gradient calculation
    net_loaded.eval() # Set the model to evaluation mode
    for data in testloader:
        # ---- Move inputs and labels to the chosen device ----
        images, labels = data[0].to(device), data[1].to(device)
        # ----------------------------------------------------
        outputs = net_loaded(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            # Ensure label and prediction are on CPU if classes is a CPU-based structure
            # or if further operations on them expect CPU tensors/values
            label_item = label.item()
            prediction_item = prediction.item()
            if label_item == prediction_item:
                correct_pred[classes[label_item]] += 1
            total_pred[classes[label_item]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy for class: plane is 50.1 %
Accuracy for class: car   is 44.1 %
Accuracy for class: bird  is 41.1 %
Accuracy for class: cat   is 17.4 %
Accuracy for class: deer  is 48.9 %
Accuracy for class: dog   is 51.2 %
Accuracy for class: frog  is 50.5 %
Accuracy for class: horse is 73.4 %
Accuracy for class: ship  is 78.0 %
Accuracy for class: truck is 75.9 %
